![model](image/model.png)

**第一步：自动计算掩码。**

​	首先将x0加噪。将[0,T]转化成[0,1]，文章通过实验得出，加噪到0.5效果不错。 然后对这个噪声图进行去噪，第一个预测的噪声的文本信息是原图，第二个是询问。两个噪声的差值得到掩码图。

​	**第二步：用DDIM进行编码**

​	用 DDIM进行编码，不是编码到完全噪声，而是编码到某个中间步骤。文本条件为空。 编码程度用r控制，r越大，越贴近文本，但是就可能与输入图像偏差更大。

​	**第三步：用掩码进行解码**

​	这里就是常规做法了。掩码的部分只由扩散模型提供。非掩码的部分用原图加噪提供，这里用的是DDIM编码时的图。 这里是DDIM的前向和后向结合，比较一致。以往的做法是直接加噪声然后去噪，就失去了一致 性。效果不好。

​	总结来说，就是提供了一种通过去噪的差计算掩码的方式。有了掩码之后，就是通过DDIM前向和反向来进行修改。DDIM前向到某一个步骤，再反向，反向的过程用DDIM的加噪，掩码部分用扩散模型。

#### 导入相关包

In [ ]:
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, DDIMScheduler
import torch
from torchvision import transforms as tfms
from tqdm.auto import tqdm
from PIL import Image
from IPython import display
import logging
from pathlib import Path
from huggingface_hub import notebook_login

logging.disable(logging.WARNING)  #禁用警告信息

if not (Path.home()/'.huggingface'/'token').exists(): 
    notebook_login()